# LISTA 7 - OBTENCIÓN DE DATOS

## <ins>Ejercicios Obligatorios </ins>

## Instalación e importación de librerías necesarias

In [ ]:
!pip install Faker #es una herramienta para generar datos falsos de forma automática dentro de una aplicación
!pip install PyMySQL #permite la interacción con bases de datos MySQL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
import codecs
from datetime import date
from datetime import datetime
from faker import Faker
import pymysql
import random
import requests
import csv
import pandas as pd
from sqlalchemy import create_engine

## Inicialización de MySQL con datos de prueba

Constantes utilizadas durante la construcción del dataset:

In [ ]:
NUMERO_CLIENTES = 500
NUMERO_PROVEEDORES = 10
SEMILLA_ALEATORIA_GENERADOR = 10
SEMILLA_ALEATORIA_RANDOM = 1

Inicializamos generador de contenido ficticio en español y semillas aleatorias para que el dataset generado sea siempre el mismo:

In [ ]:
Faker.seed(SEMILLA_ALEATORIA_GENERADOR)
random.seed(SEMILLA_ALEATORIA_RANDOM)
fake = Faker(['es_ES'])

Funciones para la generación del conjunto de datos:

In [ ]:
def build_providers_dataset(number):
  providers = []
  for i in range(1, number+1):
    providers.append({
      "provider_id": i,
      "name": fake.company(),
      "email": fake.company_email(),
      "webpage": fake.domain_name()
    })

  return {
      "providers": providers
  }


In [ ]:
def build_products_dataset(providers_info):
  products = []
  url = 'https://drive.google.com/uc?export=view&id=1D9MY0au4b7SXwhUdm6TNfsKfYzdkbAh_'
  content = requests.get(url)
  text = codecs.iterdecode(content.iter_lines(), 'utf-8')
  reader = csv.DictReader(text, delimiter=',', quotechar='"')
  for row in reader:
    products.append(row)

  categories = sorted(set([product['category'] for product in products]))
  categories = [{"category_id": i+1, "name": category} for (i, category) in enumerate(categories)]
  categories_by_name = {category["name"]: category["category_id"] for category in categories}
  products = [{"product_id": i+1,
              "name": product["name"],
              "price": float(product["price"]),
              "category_id": categories_by_name[product["category"]],
              "provider_id": random.choice(providers_info)["provider_id"]}
              for (i, product) in enumerate(products)]
  return {
      'products': products,
      'categories': categories
  }

In [ ]:
def build_people_dataset(number):

  people = []
  addresses = []
  payment_info = []
  address_id = 0
  payment_id = 0

  for i in range(1, number+1):
    # Person data
    people.append({
      "person_id": i,
      "first_name": fake.first_name(),
      "last_name": fake.last_name(),
      "birth_date": fake.date_between_dates(datetime(1960, 1, 1), datetime(2002, 6, 1)),
      "email": fake.email(),
      "phone": fake.phone_number(),
      "username": fake.user_name(),
      "password": fake.sha256(),
      "job": fake.job()
    })

    # Payment information
    if random.choice([False]*1 + [True]*2):
      payment_id += 1
      payment_info.append({
          "payment_id": payment_id,
          "person_id": i,
          "expiration": fake.credit_card_expire(),
          "number": fake.credit_card_number(),
          "provider": fake.credit_card_provider(),
          "security_code": fake.credit_card_security_code()
      })

    # Registered addresses
    for j in range(random.choice([1]*43 + [2]*6 + [3])):
      address_id+=1
      addresses.append(
      {
        "address_id": address_id,
        "person_id": i,
        "city": fake.city(),
        "number": fake.building_number(),
        "country": "España",
        "zipcode": fake.postcode(),
        "street": fake.street_name()
      })

  return {
      "people": people,
      "addresses": addresses,
      "payment_information": payment_info,
  }


In [ ]:
def build_network_dataset(people_info):

  WEB_PAGES = [fake.uri_path() for i in range(0,100)]
  ACCESS_METHOD_PROPORTION = ['GET'] * 10 + ['POST']
  pages = []
  accesses = []
  access_id = 0

  for i in range(0, len(WEB_PAGES)):
    pages.append({
        "page_id": i+1,
        "path": WEB_PAGES[i]
    })

  for person in people_info:
    # Access to webpages
    for j in range(int(random.gauss(60, 40))):
      access_id += 1
      accesses.append({
          "access_id": access_id,
          "person_id": person["person_id"],
          "method": random.choice(ACCESS_METHOD_PROPORTION),
          "ip": fake.ipv4_public(),
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
          "page_id": random.randint(1, len(WEB_PAGES)-1)
      })

  # Anonymous access
  for i in range(int(random.gauss(1000, 100))):
    access_id += 1
    accesses.append({
        "access_id": access_id,
        "person_id": None,
        "method": random.choice(ACCESS_METHOD_PROPORTION),
        "ip": fake.ipv4_public(),
        "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
        "page_id": random.randint(1, len(WEB_PAGES)-1)
    })

  return {
    "web_pages":  pages,
    "accesses": accesses
  }


In [ ]:
def build_shopping_dataset(people, products, people_addresses):

  shopping_carts = []
  shopping_cart_products = []
  orders = []
  order_products = []
  invoices = []
  cart_id = 0
  shopping_cart_id = 0
  order_id = 0
  order_product_id = 0
  invoice_id = 0

  PRODUCTS_PROBABILITY = [1]*2 + [2] * 3 + [3] * 3 + [4]*2 + [5]
  ORDER_PROBABILITY = [0]+[1]*7+[2]*3+[3]*3+[4]*2+[5]
  QUANTITY_PROBABILITY = [1]*5 +[2]*2 +[3]
  RATING_PROBABILITY = [1]+[2]+[3]*2+[4]*4+[5]*3

  for person in people:
    # Build shopping cart
    if random.choice([False * 9] + [True]):
      cart_id += 1
      shopping_carts.append({
          "cart_id": cart_id,
          "person_id": person["person_id"],
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
      })

      chosen = random.sample(products, k = random.choice(PRODUCTS_PROBABILITY))
      for product in chosen:
        shopping_cart_id += 1
        shopping_cart_products.append({
            "cart_id": cart_id,
            "product_id": product["product_id"],
            "quantity": random.choice(QUANTITY_PROBABILITY)
        })

    # Build orders
    for i in range(0, random.choice(ORDER_PROBABILITY)):
      order_id += 1
      order_price = 0
      chosen = random.sample(products, k = random.choice(PRODUCTS_PROBABILITY))
      for product in chosen:
        order_product_id += 1
        quantity = random.choice(QUANTITY_PROBABILITY)
        order_products.append({
            "order_id": order_id,
            "product_id": product["product_id"],
            "quantity": quantity
        })
        order_price += quantity * product['price']

      person_addresses = [address for address in people_addresses if address["person_id"] == person["person_id"]]
      delivery_address = random.choice(person_addresses)
      billing_address = random.choice(person_addresses)
      orders.append({
          "order_id": order_id,
          "person_id": person["person_id"],
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
          # Purposely left wrong
          "delivery_address": delivery_address['address_id'],
          "billing_address": billing_address['address_id'],
          "price": order_price
      })

  # Build invoices
  for order in random.choices(orders, k = int(len(orders) * 0.8)):
    invoice_id += 1
    invoices.append({
      "invoice_id": invoice_id,
      "order_id": order["order_id"],
      "date": fake.date_time_between(order["date"], datetime(2020,9,1,23,59,59)),
      "rating": random.choice(RATING_PROBABILITY)
    })

  return {
      'carts': shopping_carts,
      'cart_product': shopping_cart_products,
      'orders': orders,
      'order_product': order_products,
      'invoices': invoices
  }

Construcción incremental del dateset:

In [ ]:
dataset = {}
dataset.update(build_providers_dataset(NUMERO_PROVEEDORES))
dataset.update(build_products_dataset(dataset['providers']))
dataset.update(build_people_dataset(NUMERO_CLIENTES))
dataset.update(build_network_dataset(dataset['people']))
dataset.update(build_shopping_dataset(dataset['people'], dataset['products'], dataset['addresses']))

### Carga de dataset en MySQL

#### Creación de la base de datos

Script para la creación de la base de datos en mysql

```
DROP SCHEMA IF EXISTS shop;
CREATE SCHEMA shop CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
USE shop;

DROP TABLE IF EXISTS accesses;
CREATE TABLE accesses (
    access_id INT,
    person_id INT NULL DEFAULT NULL,
    date DATETIME,
    ip VARCHAR(20),
    method VARCHAR(10),
    page_id INT,
    PRIMARY KEY(access_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS web_pages;
CREATE TABLE web_pages (
    page_id INT,
    path VARCHAR(250),
    PRIMARY KEY(page_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS carts;
CREATE TABLE carts (
    cart_id INT,
    person_id INT,
    date DATETIME,
    PRIMARY KEY(cart_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS cart_product;
CREATE TABLE cart_product (
    cart_id INT,
    product_id INT,
    quantity INT,
    PRIMARY KEY(cart_id, product_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS categories;
CREATE TABLE categories (
    category_id INT,
    name VARCHAR(100),
    PRIMARY KEY(category_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS invoices;
CREATE TABLE invoices (
    invoice_id INT,
    order_id INT,
    date DATETIME,
    rating INT,
    PRIMARY KEY(invoice_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS orders;
CREATE TABLE orders (
    order_id INT,
    person_id INT,
    date DATETIME,
    billing_address INT,
    delivery_address INT,
    price DECIMAL(18,6),
    PRIMARY KEY(order_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS order_product;
CREATE TABLE order_product (
    order_id INT,
    product_id INT,
    quantity INT,
    PRIMARY KEY(order_id, product_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS payment_information;
CREATE TABLE payment_information (
    payment_id INT,
    person_id INT,
    number VARCHAR(30),
    provider VARCHAR(200),
    security_code VARCHAR(10),
    expiration VARCHAR(5),
    PRIMARY KEY(payment_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS people;
CREATE TABLE people (
    person_id INT,
    birth_date DATETIME,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(150),
    job VARCHAR(100),
    phone VARCHAR(20),
    username VARCHAR(50),
    password VARCHAR(100),
    PRIMARY KEY(person_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS addresses;
CREATE TABLE addresses (
    address_id INT,
    person_id INT,
    city VARCHAR(30),
    country VARCHAR(20),
    number INT,
    street VARCHAR(100),
    zipcode INT,
    PRIMARY KEY(address_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_id INT,
    category_id INT NULL DEFAULT NULL,
    provider_id INT NULL DEFAULT NULL,
    name VARCHAR(200),
    price DECIMAL(10,4),
    PRIMARY KEY(product_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

DROP TABLE IF EXISTS providers;
CREATE TABLE providers (
    provider_id INT,
    name VARCHAR(50),
    email VARCHAR(100),
    webpage VARCHAR(100),
    PRIMARY KEY(provider_id)
)
CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

COMMIT;
```

Cargamos los datos del dataset en la base de datos

In [ ]:
con = pymysql.connect(host='localhost', user='admin',password='Password0', database='shop')
try:
    for table in dataset:
      first_time = True
      sql = ""
      with con.cursor() as cur:
        for entity in dataset[table]:
          if (first_time):
            first_time = False
            str_columns = ",".join(entity.keys())
            str_values = ",".join(["%s"] * len(entity.keys()))
            sql = f"INSERT INTO {table} ({str_columns}) VALUES ({str_values})"
          cur.execute(sql, tuple(entity.values()))
        con.commit()
finally:
    con.close()

OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([Errno 99] Cannot assign requested address)")

Script para las restriciones de foreign key de la base de datos

```
ALTER TABLE products ADD CONSTRAINT FK_product_category_id
FOREIGN KEY(category_id) REFERENCES categories(category_id)
ON DELETE SET NULL
ON UPDATE CASCADE;

ALTER TABLE products ADD CONSTRAINT FK_product_provider_id
FOREIGN KEY(provider_id) REFERENCES providers(provider_id)
ON DELETE SET NULL
ON UPDATE CASCADE;

ALTER TABLE addresses ADD CONSTRAINT FK_addresses_person_id
FOREIGN KEY(person_id) REFERENCES people(person_id)
ON DELETE CASCADE
ON UPDATE CASCADE;

ALTER TABLE payment_information ADD CONSTRAINT FK_payment_information_person_id
FOREIGN KEY(person_id) REFERENCES people(person_id)
ON DELETE CASCADE
ON UPDATE CASCADE;

ALTER TABLE order_product ADD CONSTRAINT FK_order_product_order_id
FOREIGN KEY(order_id) REFERENCES orders(order_id)
ON DELETE CASCADE
ON UPDATE CASCADE;

ALTER TABLE order_product ADD CONSTRAINT FK_order_product_product_id
FOREIGN KEY(product_id) REFERENCES products(product_id)
ON DELETE RESTRICT
ON UPDATE CASCADE;

ALTER TABLE orders ADD CONSTRAINT FK_orders_person_id
FOREIGN KEY(person_id) REFERENCES people(person_id)
ON DELETE RESTRICT
ON UPDATE CASCADE;

ALTER TABLE orders ADD CONSTRAINT FK_orders_billing_address_id
FOREIGN KEY(billing_address) REFERENCES addresses(address_id)
ON DELETE RESTRICT
ON UPDATE CASCADE;

ALTER TABLE orders ADD CONSTRAINT FK_orders_delivery_address_id
FOREIGN KEY(delivery_address) REFERENCES addresses(address_id)
ON DELETE RESTRICT
ON UPDATE CASCADE;

ALTER TABLE accesses ADD CONSTRAINT FK_accesses_person_id
FOREIGN KEY(person_id) REFERENCES people(person_id)
ON DELETE SET NULL
ON UPDATE CASCADE;

ALTER TABLE accesses ADD CONSTRAINT FK_accesses_page_id
FOREIGN KEY(page_id) REFERENCES web_pages(page_id)
ON DELETE CASCADE
ON UPDATE CASCADE;

ALTER TABLE carts ADD CONSTRAINT FK_carts_person_id
FOREIGN KEY(person_id) REFERENCES people(person_id)
ON DELETE CASCADE
ON UPDATE CASCADE;

ALTER TABLE cart_product ADD CONSTRAINT FK_cart_product_cart_id
FOREIGN KEY(cart_id) REFERENCES carts(cart_id)
ON DELETE CASCADE
ON UPDATE CASCADE;

ALTER TABLE cart_product ADD CONSTRAINT FK_cart_product_product_id
FOREIGN KEY(product_id) REFERENCES products(product_id)
ON DELETE RESTRICT
ON UPDATE CASCADE;

ALTER TABLE invoices ADD CONSTRAINT FK_invoices_order_id
FOREIGN KEY(order_id) REFERENCES orders(order_id)
ON DELETE RESTRICT
ON UPDATE RESTRICT;
```

## EJERCICIO 1

La base de datos shop tiene una tabla people con información sobre los clientes de la tienda ficticia. Escribe el código Python necesario para inicializar un DataFrame con el contenido de la tabla:

## EJERCICIO 2

La siguiente URL https://datosabiertos.carm.es/odata/Agricultura/IMIDA_dia_2018.csv contiene el informe meteorológico diario de las diferentes estaciones meteorológicas de la Región de Murcia a lo largo del año 2018. Observa el contenido del fichero csv y a continuación utiliza la función read_csv de pandas sobre esta URL con los  parámetros necesarios (header, sep, decimal, quotechar y encoding).

In [ ]:
# Importa la biblioteca pandas, que es útil para el análisis y manipulación de datos.
import pandas as pd

# Lee un archivo CSV desde una URL y lo carga en un DataFrame de pandas.
meteo_df = pd.read_csv(
    "https://datosabiertos.carm.es/odata/Agricultura/IMIDA_dia_2018.csv ",  # URL del archivo CSV a leer.
    header=0,  # Especifica que la fila número 0 contiene los nombres de las columnas.
    sep=";",  # Define el carácter separador de campos como punto y coma (;) en lugar de la coma, que es la opción por defecto.
    decimal=".",  # Indica que el punto (.) se utiliza como separador decimal en los números.
    quotechar="\"",  # Especifica que las cadenas de texto están encerradas entre comillas dobles.
    encoding='latin-1'  # Establece la codificación del archivo como 'latin-1' (ISO-8859-1), que es común en archivos de texto.
)

# Muestra el DataFrame cargado. Esto generalmente mostrará las primeras filas del DataFrame.
meteo_df

,FECHA,EST,MUNICIPIO,PARAJE,TMED,TMAX,TMIN,HRMED,HRMAX,HRMIN,PREC,RADMED,VVMED,VVMAX,DVMED,ETO
0,01/01/18,AL31,Totana,Lebor,"12,87","19,17","5,909","46,44","75,9","22,91",0,"124,03","2,05","6,821","249,66","2,23"
1,02/01/18,AL31,Totana,Lebor,"13,05","20,99","8,2","44,91","64,69","23,74",0,"122,8","1,69","8,9","272,61","2,24"
2,03/01/18,AL31,Totana,Lebor,"13,57","22,57","7,06","61,66",81,"37,48",0,"124,63","0,85","3,077","13,25","1,34"
3,04/01/18,AL31,Totana,Lebor,"14,52","23,84","8,14","63,91","88,6","33,87",0,"124,25","1,12","4,753","291,87","1,67"
4,05/01/18,AL31,Totana,Lebor,"10,36","17,7","4,565","77,86","99,9","44,64",0,"125,46","0,99","3,665","260,43","1,13"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17742,27/12/18,TP91,Torre Pacheco,Torre Pacheco,"10,76","16,03","7,39","88,28","99,35","65,07","0,2","93,96","0,74","3,626","266,05","0,79"
17743,28/12/18,TP91,Torre Pacheco,Torre Pacheco,"11,19","15,81","6,6","83,62","96,95","58,85",0,"93,32","0,66","3,724","320,25","0,8"
17744,29/12/18,TP91,Torre Pacheco,Torre Pacheco,"12,23","14,68","10,19","91,24",98,"73,23","9,2","35,73","1,35","5,978","15,21","0,81"
17745,30/12/18,TP91,Torre Pacheco,Torre Pacheco,"11,01","15,29","6,7","85,61","96,72","62,92","0,2","108,46","1,13","4,802","15,22","0,92"


## EJERCICIO 3

El Instituto de Salud Carlos III ofrece gratuitamente información actualizada sobre la situación del COVID-19 en España. En la siguiente URL podemos obtener un fichero csv actualizado con los casos positivos notificados por las Comunidades Autónomas a nivel provincial: https://cnecovid.isciii.es/covid19/resources/casos_diagnostico_provincia.csv. En el siguiente enlace se nos describe este conjunto de datos y se nos proporcionan otros conjuntos de datos de interés.

Utiliza el método read_csv con los parámetros adecuados para obtener el DataFrame con los casos positivos por provincia. Muestra a continuación su contenido:

In [ ]:
# Lee un archivo CSV desde una URL y lo carga en un DataFrame de pandas.
casos_df = pd.read_csv(
    "https://cnecovid.isciii.es/covid19/resources/casos_diagnostico_provincia.csv",  # URL del archivo CSV a leer.
    header=0,  # Especifica que la fila número 0 contiene los nombres de las columnas.
    sep=",",  # Define el carácter separador de campos como coma (,) que es la opción por defecto en CSV.
    decimal=".",  # Indica que el punto (.) se utiliza como separador decimal en los números.
    quotechar="\"",  # Especifica que las cadenas de texto están encerradas entre comillas dobles.
    encoding="utf-8"  # Establece la codificación del archivo como 'utf-8', que es común y soporta caracteres especiales.
)

# Muestra el DataFrame cargado. Esto generalmente mostrará las primeras filas del DataFrame.
casos_df

,provincia_iso,fecha,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_ag,num_casos_prueba_elisa,num_casos_prueba_desconocida
0,A,2020-01-01,1,1,0,0,0,0
1,AB,2020-01-01,0,0,0,0,0,0
2,AL,2020-01-01,0,0,0,0,0,0
3,AV,2020-01-01,0,0,0,0,0,0
4,B,2020-01-01,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...
20718,V,2021-01-25,85,70,0,15,0,0
20719,VA,2021-01-25,0,0,0,0,0,0
20720,VI,2021-01-25,0,0,0,0,0,0
20721,Z,2021-01-25,0,0,0,0,0,0


## EJERCICIO 4

La siguiente URL contiene información de películas estadounidenses obtenidas de la Wikipedia en formato JSON: https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json. Utiliza el método `read_json` de `pandas` para cargar su contenido en un DataFrame:

In [ ]:
# Importa la biblioteca pandas, que es útil para el análisis y manipulación de datos.
import pandas as pd

# Lee un archivo JSON desde una URL y lo carga en un DataFrame de pandas.
peliculas_df = pd.read_json("https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json")

# Muestra el DataFrame cargado. Esto generalmente mostrará las primeras filas del DataFrame.
peliculas_df

,title,year,cast,genres,href,extract,thumbnail,thumbnail_width,thumbnail_height
0,After Dark in Central Park,1900,[],[],None,NaN,NaN,NaN,NaN
1,Boarding School Girls' Pajama Parade,1900,[],[],None,NaN,NaN,NaN,NaN
2,Buffalo Bill's Wild West Parad,1900,[],[],None,NaN,NaN,NaN,NaN
3,Caught,1900,[],[],None,NaN,NaN,NaN,NaN
4,Clowns Spinning Hats,1900,[],[Silent],Clowns_Spinning_Hats,Clowns Spinning Hats is a black-and-white sile...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
36268,Aquaman and the Lost Kingdom,2023,"[Jason Momoa, Amber Heard, Willem Dafoe, Patri...",[Superhero],Aquaman_and_the_Lost_Kingdom,Aquaman and the Lost Kingdom is an upcoming Am...,https://upload.wikimedia.org/wikipedia/en/thum...,320.0,163.0
36269,Untitled Ghostbusters: Afterlife sequel,2023,"[Mckenna Grace, Carrie Coon, Finn Wolfhard, Pa...","[Comedy, Supernatural]",Untitled_Ghostbusters:_Afterlife_sequel,The untitled Ghostbusters: Afterlife sequel is...,NaN,NaN,NaN
36270,Rebel Moon,2023,"[Sofia Boutella, Charlie Hunnam, Ray Fisher, D...",[Science Fiction],Rebel_Moon,Rebel Moon is an upcoming American epic space ...,NaN,NaN,NaN
36271,Migration,2023,[],[],Migration_(2023_film),This is a list of productions produced by Illu...,NaN,NaN,NaN


## EJERCICIO 5

Haciendo uso de la librería `requests` y `BeautifulSoup`, accede a la siguiente URL https://catalogoreina.com/859-grifos-cocina-roca y recupera el nombre de los artículos mostrados:

In [ ]:
# Importa la clase BeautifulSoup de la biblioteca bs4 para el análisis de HTML y XML
from bs4 import BeautifulSoup
# Importa la biblioteca requests para hacer solicitudes HTTP
import requests

# Define la URL de la página web que se va a analizar
URL = "https://catalogoreina.com/859-grifos-cocina-roca"
# Realiza una solicitud GET a la URL y almacena la respuesta en la variable r
r = requests.get(URL)
# Crea un objeto BeautifulSoup a partir del contenido HTML de la respuesta
html_soup = BeautifulSoup(r.text, 'html.parser')  # Especifica el parser HTML para evitar advertencias

# Busca todos los elementos <a> que tienen la clase 'product-name' en el HTML
tags = html_soup.find_all(name='a', attrs={'class': 'product-name'})
# Itera sobre cada etiqueta encontrada
for tag in tags:
    # Imprime el valor del atributo "title" de cada etiqueta
    print(tag.get("title"))

Cala grifo de cocina negro mate caño giratorio Roca A5A846ENB0
CALA Grifo diseño para cocina con caño giratorio Roca A5A846EC00
CALA grifo monomando de cocina con caño giratorio en negro Roca A5A836ENB0
CALA Mezclador para cocina con caño giratorio Roca A5A856EC00
CALA Monomando de Cocina Roca A5A836EC00. Ofertas de grifos de cocina
CARMEN Bimando cocina A5A844BC00 Roca
Exclusivo CALA grifo para cocina Negro de Roca A5A856ENB0 calidad
GLERA diseño ergonómico monomando cocina moderno Roca A5A834DC00
GLERA Grifo fregadero extraíble diseño elegante para cocina A5A814DC00 Roca
GLERA Mezclador monomando para cocina con caño giratorio Roca A5A844DC00
GLERA Monomando para cocina con caño giratorio y ducha Roca A5A854DC00
GLERA Pro - Mezclador monomando para cocina con caño giratorio con muelle Roca A5A8A4DC00


## EJERCICIO 6

Haciendo uso de la librería `requests` y `BeautifulSoup`, accede a la página web del DB-Engines https://db-engines.com/en/ranking y recupera los nombres de las 10 bases de datos más populares:

PISTA: el selector CSS [`nth-child`](https://developer.mozilla.org/es/docs/Web/CSS/:nth-child) puede serte de utilidad.

OTRA PISTA: para evitar que salga el contenido de la etiqueta `info` junto al nombre de la base de datos, haz uso del método [`extract`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract).

In [ ]:
# Define la URL de la página web que se va a analizar
URL = "https://db-engines.com/en/ranking"
# Realiza una solicitud GET a la URL y almacena la respuesta en la variable r
r = requests.get(URL)
# Crea un objeto BeautifulSoup a partir del contenido HTML de la respuesta
html_soup = BeautifulSoup(r.text, 'html.parser')  # Especifica el parser HTML para evitar advertencias

# Utiliza un selector CSS para encontrar los elementos <a> dentro de la cuarta columna de las filas <th> de la tabla
# Se limita a obtener solo los primeros 10 elementos
nombres = html_soup.select("th.pad-l:nth-child(4) a", limit=10)

# Itera sobre cada elemento encontrado en la lista nombres
for nombre in nombres:
    # Obtiene el texto del elemento
    resultado = nombre.get_text()
    # Imprime el texto del resultado, eliminando la parte "Detailed" si está presente
    print(resultado.split("Detailed")[0])

Oracle 
MySQL 
Microsoft SQL Server 
PostgreSQL 
MongoDB 
Redis 
Snowflake 
Elasticsearch
IBM Db2
SQLite 


## EJERCICIO 7

Haciendo uso de la librería `requests` y `BeautifulSoup`, recupera el nombre, los acabados, las medidas y el plazo de entrega del siguiente artículo: https://catalogoreina.com/nuestras-marcas-muebles-bano/10233-mueble-bano-con-patas-althea-moderno-3-cajones.html

In [ ]:
# Define la URL de la página del producto que se va a analizar
URL = "https://catalogoreina.com/nuestras-marcas-muebles-bano/10233-mueble-bano-con-patas-althea-moderno-3-cajones.html"
# Realiza una solicitud GET a la URL y almacena la respuesta en la variable r
r = requests.get(URL)
# Crea un objeto BeautifulSoup a partir del contenido HTML de la respuesta
html_soup = BeautifulSoup(r.text, 'html.parser')  # Especifica el parser HTML para evitar advertencias

# Busca el elemento <h1> en la página, que normalmente contiene el nombre del producto
nombre = html_soup.find(name="h1")
# Imprime el texto del nombre del producto, eliminando espacios adicionales
print(nombre.get_text(strip=True))

# Busca el div que contiene la descripción corta del producto utilizando su ID
descripcion = html_soup.find(name="div", attrs={'id': 'short_description_content'})
# Encuentra todos los elementos <p> dentro de la descripción corta
descripcion_p = descripcion.find_all(name="p")

# Inicializa una lista vacía para almacenar los párrafos de la descripción
des_short = []
# Itera sobre cada párrafo encontrado
for p in descripcion_p:
    # Añade el texto de cada párrafo a la lista, eliminando espacios adicionales
    des_short.append(p.get_text(strip=True))

# Imprime el primer párrafo de la descripción corta
print(des_short[0])
# Imprime el segundo párrafo de la descripción corta
print(des_short[1])

# Busca el tercer párrafo dentro del contenedor de la descripción del producto usando un selector CSS
plazos = html_soup.select_one('.page-product-box .rte p:nth-child(3)')
# Imprime el texto del párrafo encontrado, eliminando espacios adicionales
print(plazos.get_text(strip=True))

... Muebles de baño ALTHEA de 3 cajones.
Acabados:Blanco Brillo, Antracita y Haya.
Medidas:60 - 70 - 80 - 100  x 82 x 45 cm.
Plazo de entrega:De 12 a 18 días hábiles


## <ins>Ejercicios Opcionales </ins>

## EJERCICIO 8

Esta base de datos también contiene una tabla orders con información de la cabecera de los pedidos de la tienda. En esta tabla existe una columna price que almacena el importe total del pedido. Construye a continuación un DataFrame con las cabeceras de pedidos ordenada por importe de manera descendente:

## EJERCICIO 9

La base de datos también contiene las tablas web_pages y accesses con información de las páginas web de la empresa y de los accesos realizados a ellas respectivamente. Ambas tablas pueden relacionarse por el campo page_id. Inicializa un DataFrame que contenga los campos page_id y path de web_pages y el total de accesos realizado sobre cada una de ellas:

## EJERCICIO 10

Crea un diccionario Python que represente una factura en formato JSON:

* Contendrá 4 campos cuyos valores puedes inventarte, pero siguiendo las siguientes indicaciones:
  * **client_id**: identificador de cliente, de tipo cadena.
  * **products**: array con al menos dos productos de tipo objeto. Cada objeto tendrá dos campos:
    * **name**: de tipo cadena.
    * **price**: de tipo numérico.
  * **date**: de tipo cadena.
  * **address**: de tipo objeto, con los siguientes campos:
    * **street**: de tipo objeto, con los siguientes campos:
      * **name**: de tipo cadena.
      * **number**: de tipo numérico.
    * **zipcode**: de tipo numérico.

El diccionario tendrá que tener un formato tal que sea aceptado por el siguiente [validador](https://jsonlint.com/) de contenido JSON.

## EJERCICIO 11

Utilizando la librería PyPDF4, recupera el número de páginas del siguiente PDF:
```
PDF_URL = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_278_COVID-19.pdf'
```

## EJERCICIO 12

Utilizando la librería PyPDF4, recupera la fecha de creación del documento (campo /CreationDate de los metadatos):

## EJERCICIO 13

Extrae la información del pdf en tablas y obten aquella que contiene los Detalles de los quince países con más casos confirmados fuera de Europa.

PISTA: esta tabla es la última del documento PDF. Utiliza la función len para obtener el total de tablas extraídas y saber cuál de ellas seleccionar.

## EJERCICIO 14

Mediante `read_excel`, carga el contenido de todas las hojas del fichero excel "Orders-With Nulls.xlsx", muestra el nombre de todas las hojas y muestra la hoja denominada `Summary`: